In [2]:
#import modules
from fuzzywuzzy import fuzz
import os

In [3]:
#Open database to extract publications only
with open("../Datataxa_extraction/FIRSTTEST.csv", mode="r") as file:
    lines = file.readlines()
    publications = []
    for line in lines:
        #Split line in cells
        cells = line.split("\",\"")
        #If line contains more than 2 cells (third one contains publication info) extract it
        if len(cells) >= 2:
            publications_cell = cells[2]
            publications.extend(publications_cell.split("|"))
    #Remove duplicates and sort them alphabetically
    unique_publications = sorted(list(set(publications)))

#Print values
print(f"Number of elements in publications: {len(publications)}")
print(f"Number of unique elements in publications: {len(unique_publications)}")

Number of elements in publications: 62144
Number of unique elements in publications: 12564


In [4]:
import time

In [23]:
#Remove very similar titles
# copy list
publications_to_filter = unique_publications.copy()

#test
# publications_to_filter =[
#     "A phylogenetic analysis of the Mimosoideae (Leguminosae) based on chloroplast sequence data",
#     "A Diploids-First Approach to Species Delimitation and Interpreting Polyploid Evolution in the Fern Genus Astrolepis (Pteridaceae)",
#     "A New Phylogeny for the Grasses",
#     "A diploids-first approach to species delimitation and interpreting polyploid evolution in the fern genus Astrolepis (Pteridaceae)",
#     "A phylogenetic analysis of the Mimosoideae (Leguminosae) based on chloroplast DNA sequence data",
# ]

#check if files exist and remove them
removed_file = "removed_titles.txt"
if os.path.exists(removed_file):
    os.remove(removed_file)

conserved_file =  "conserved_titles.txt"
if os.path.exists(conserved_file):
    os.remove(conserved_file)

#open file to save removed titles for double check
with open(removed_file, "a") as file:
    # doble iteration to check all values
    for i, t1 in enumerate(publications_to_filter):
        if t1:
            for j, t2 in enumerate(publications_to_filter[i:]):
                print(f"Progress: {i} of {len(publications_to_filter)}", end="\r")
                #get distance between titles
                if t2:
                    distance = fuzz.token_sort_ratio(t1.lower(), t2.lower())
                    #check that is not the same title
                    if i != j+i:
                        #if distance is greater than remove this title
                        if distance > 95:
                            file.write(f"{t1}\n{t2}\t{distance}\n\n")
                            publications_to_filter[j+i] = None

#purge Nones from list
ratio_publications = [i for i in publications_to_filter if i != None]
with open(conserved_file, "w") as file:
    file.write("\n".join(ratio_publications))
print(f"Number of unique elements in publications after similarity filter: {len(ratio_publications)}")

Number of unique elements in publications after similarity filter: 12169


In [ ]:
#Export list in a html format  or csv to import in google sheets to contain a scholar link (and maybe a google link as second choice) like:
#https://scholar.google.com/scholar?q=Molecular+phylogeny+and+character+evolution+of+Anemiaceae+%28Schizaeales%29